# ニューラルネットワーク

## 1. はじめに
ニューラルネットワークは，脳の神経回路網をモデル化した機械学習の手法である．近年，深層学習（Deep Learning）の登場により再びニューラルネットワークが注目を集めている．深層学習は，画像認識，音声認識，自然言語処理など様々な分野で取り入られており，Google社をはじめ， 多くの企業で利用，研究がされている．本資料では，ニューラルネットワークの基本的な原理， 学習方法について記述する．

## 2. 基本構造
ニューラルネットワークは，複数のニューロンとニューロン同士を結合するシナプスから構成される.  
シナプスは各結合の強さを表す結合重みを持っている.  
ニューラルネットワークの図を示す.  

<img src="img/neuralnetwork.png" width="400">
<div style="text-align: center;">
図1.  3層の階層型ニューラルネットワーク
</div>

ニューラルネットワークは，とても複雑な関数だと思うと理解が早い．  
関数なので，入力$x$と，その関数$f(x)$の出力$y$をとる．  
さらに，この関数$f(x)$は，合成関数となっており，イメージとしては，$f(x) = u(s(g(x)+h(x)) + t(m(x)+n(x)) )$のように複雑な合成関数である．  
各関数にはパラメータがあり，これをうまく調整すると，ある入力$x$に対して，所望の$y$が出力するようになる．  
この調整をトレーニングデータから自動的に行うのがニューラルネットワークである．  

<img src="img/system.png" width="400">
<div style="text-align: center;">
図2.  ニューラルネットワークのイメージ
</div>

ニューラルネットワークは，入力層，中間層，出力層の多層構造になっており，各ニューロンは前の層のニューロンの出力を入力として受け取り，計算を行ったあと，次のニューロンへと出力する．  
各ニューロンの出力が次々と伝播していき，出力層のニューロンからネットワークの出力を得る．  
単体のニューロンの構造を図に示す．  

<img src="img/neuron.png" width="400">
<div style="text-align: center;">
図3.   単体のニューロン
</div>

ニューロンは複数の入力を受け取り，1つの値を出力する．  
ある層の上位ニューロン$j$は，その前の層の$I$個の下位ニューロン出力$o_i$を入力として受け取る．  
このとき，ニューロン間の重み$w_ij$と入力$o_i$の積和計算する．  
そして，閾値$\theta_j$を加えた値を重み付き和$X_j$として算出する．  
その重み付き和の値を活性化関数$f(x)$の引数として，活性化関数の値をニューロン$j$の出力とする．  
これらを式で表すと以下のようになる．  

$$
X_j =  \sum_{i=1}^{I} w_{ij} o_i + \theta_j \\
o_j =  f(X_j)
$$

活性化関数には，以下のシグモイド関数や双曲線正接関数などが使われる．

$$
f(x) = \frac{1}{1 + \exp(-\epsilon x)} \\
f(x) = \tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}
$$

<img src="img/sigmoid.png" width="400">
<div style="text-align: center;">
図4.   シグモイド関数と双曲線正接関数のグラフ
</div>

ここで$\epsilon$はシグモイド関数の$x=0$のときの傾きを表している．  
$\epsilon$が小さいほど傾きが緩やかになり，大きいほど傾きが傾きが大きくなる．  
傾きが急なシグモイド関数ほど，少しの変化で0あるいは1の出力となるので，反応が敏感な関数であると言える．

<img src="img/sigmoid2.png" width="400">
<div style="text-align: center;">
図5.   シグモイド関数と双曲線正接関数のグラフ
</div>

近年，これらの活性化関数の代わりに正規化線形関数(Rectified linear function)が利用されることが多くなっている．  
一般的には，ReLUと呼ばれる．  
ReLUは，

- $\max(0,x)$は単純ゆえに早い
- 0を作るので，スパース性につながる
- $x>0$の部分では微分値が常に1であるため勾配消失の心配はない  

という利点を持つ．  
ReLUの式とグラフを以下に示す．  

$$
f(x) = \max(0,x)
$$

<img src="img/relu.png" width="400">
<div style="text-align: center;">
図6.   ReLU関数
</div>

以上の計算を入力層側のニューロンから順に行う．  
ただし，入力層のニューロンはこれらの計算を行わず，入力をそのまま出力する．  
つまり，入力層2個，中間層2個，出力層1のネットワークは，厳密は以下の図のようになる．  

<img src="img/ff1.png" width="400">
<div style="text-align: center;">
図7.  本来，入力層にはニューロンはない
</div>

しかし，イラストのバランスが悪いので，入力ニューロンは描いてネットワークを表現することが多い．

<img src="img/ff2.png" width="400">
<div style="text-align: center;">
図8.  バランスが悪いので入力ニューロンは描くことが多い
</div>

この前向きの計算により，入力層への入力に対し，中間層を経て，出力層から出力を得ることができる．

## 3. 学習の目的

ニューラルネットワークは，理想の出力に近いほど性能が高いといえる．  
各出力ニューロン$k$に対応する教師信号$t_k$が与えられたとき，各入力パターンに対する誤差$E_p$とP個の入力パターンの誤差の総和$E_{all}$は以下の式で表すことができる．

$$
E_p = \frac{1}{2}\sum_{k = 1}^{K}(o_k - t_k)^2 \\
E_{all}  =  \sum_{p = 1}^{P} E_p
$$

ニューラルネットワークの学習は，この誤差総和$E_{all}$を最小化し，入力に対して適切な出力が得られるように各結合重みと閾値を調整することである．

## 4. 仮想ニューロン

ニューラルネットワークの出力を最適化するには，各結合重みと閾値を調整すれば良いが，それぞれを修正するのは手間がかかる．  
そこで閾値を重みとして扱うために仮想ニューロンを導入する．  
入力層，中間層，出力層のニューロン数がそれぞれ$I$個，$J$個，$K$個のニューラルネットワークに仮想ニューロンを導入した形を図に示す．  

<img src="img/virtual.png" width="400">
<div style="text-align: center;">
図9.  仮想ニューロン
</div>

仮想ニューロンは出力層を除く，各層のニューロンの最後に追加する．  
図のニューラルネットワークでは，$I + 1$番目，$J + 1$番目に追加している．  
各仮想ニューロンは次の層の全ニューロンと結合しており，結合先のニューロンの閾値を結合重みとしている．  
また，各仮想ニューロンは常に$1$を出力している．これにより，重み付け和の式は以下のように置き換えられる．  

$$
\begin{eqnarray}
	X_j & = & \sum_{i = 1}^{I} w_{ij} o_i - \theta_j \nonumber \\
	& = & \sum_{i = 1}^{I} w_{ij} o_i + w_{(I + 1), j} \nonumber \\
	& = & \sum_{i = 1}^{I} w_{ij} o_i + w_{(I + 1), j} \cdot 1 \nonumber \\
	& = & \sum_{i = 1}^{I} w_{ij} o_i + w_{(I + 1), j} \cdot o_{I + 1} \nonumber \\
	& = & \sum_{i = 1}^{I + 1} w_{ij} o_i
	\end{eqnarray}
$$

この式変形により，閾値も重みとして扱うことができる．  
これにより，この後のの重みを修正する式によって，間接的に閾値も修正することができる．  
ただし，閾値がなくなったわけではないので，実装のときには注意が必要である．

## 5． 前向き計算のデモ
ここまでのところで，ニューラルネットワークの前向き計算ができるようになった．  
デモを実行して，確認してみよう．  

前向き計算の例を考えてみよう．  
例えば，魚Aと魚Bを仕分けするシステムを作りたいとする．  
ベルトコンベアの乗って流れてきた魚の体長と体重を測って，それらから魚を推定する．  
体長をx, 体重をyとすると2次元平面上にマッピングできる．  
図の左のようになる．  
これを見ると，青の魚Aと赤の魚Bが入り組んでいて，分類が難しそうである．  

一方，簡単な例として図の右のようなものを考えてみよう．  
この図では，きれいに分けることができそうである．  


<img src="img/fish1.png" width="800">
<div style="text-align: center;">
図10.  魚の分類
</div>

<img src="img/fish2.png" width="800">
<div style="text-align: center;">
図11.  魚の分類
</div>

<img src="img/fish3.png" width="800">
<div style="text-align: center;">
図12.  魚の分類
</div>

<img src="img/xor1.png" width="800">
<div style="text-align: center;">
図13.  XORの例
</div>

<img src="img/xor2.png" width="500">
<div style="text-align: center;">
図14.  XORの例
</div>

以下のリンクより， 前向き計算を試してみよう．
- [chap3_1_forward](https://colab.research.google.com/github/crotsu/Bousai_AI/blob/master/chap3_Python/chap3_1_forward)

## 5. バックプロパゲーション法